# This notebook provide a template code to go faster in classification project : numerical, images

Importation dependencies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import VotingClassifier, BaggingClassifier

# models
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
#cnn
from keras import layers
from keras import models
from keras import optimizers
#pretrained
from keras.applications import VGG16


In [ ]:
path_train = "path/train.csv"
path_test = "path/test.csv"

train = pd.read_csv(path_train)
test = pd.read_csv(path_test)
train.head(5)

Get target, values, preprocessing...

# Prepare for training

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(prepro_train,
                                                  Y_train,
                                                  test_size = 0.3, random_state = 42)
print(x_train.shape)

# Training on usual classfifiaction model 

In [ ]:
XGB = XGBClassifier()
    

XGB.fit(x_train, y_train)

predictions_xgb = XGB.predict(x_val)

print("Accuracy : ",  accuracy_score(y_val, predictions_xgb))

# Utiliser la validation croisée pour évaluer la performance du modèle
scores = cross_val_score(XGB, x_train, y_train, cv=5, scoring='accuracy')
print("Cross-validation accuracy scores: ", scores)
print("Mean cross-validation accuracy: ", scores.mean())

# CNN classification

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPool2D(2))
model.add(layers.Conv2D(64, 3, activation='relu'))
model.add(layers.MaxPool2D(2))
model.add(layers.Conv2D(128, 3, activation='relu'))
model.add(layers.MaxPool2D(2))
model . add ( layers . BatchNormalization () )
model.add(layers.Dropout(0.2))
model . add ( layers . Flatten () )
model . add ( layers . Dense ( units =64 , activation = "relu" ) )
model . add ( layers . Dense ( units =10 , activation = "softmax" ) )

print(model.summary())

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])
history=model.fit(X_train, digit,validation_split=0.2, epochs=1)

# Fine tuning

In [ ]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3))

conv_base.summary()
##########################################################

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary

##########################################################

# we only activate the block 5

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False
conv_base.summary

##########################################################

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.RMSprop(1e-5),
              metrics=['accuracy'])

history=model.fit(X_train_tune, digit,validation_split=0.2,
                  epochs=10, batch_size=32)


# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

def bag_model(model, x_train, y_train, x_val, y_val, cross_val):
    bag = BaggingClassifier(estimator=model,
                            n_estimators=10, 
                            random_state=0).fit(x_train, y_train)

    predictions_bag = bag.predict(x_val)

    print("accuracy bag: ",  accuracy_score(y_val, predictions_bag))

    if cross_val:
        # Utiliser la validation croisée pour évaluer la performance du modèle
        scores = cross_val_score(bag, x_train, y_train, cv=5, scoring='accuracy')
        print("Cross-validation accuracy scores: ", scores)
        print("Mean cross-validation accuracy: ", scores.mean())
    return bag
# ex with xgb
bag = bag_model(XGBClassifier(random_state=42), x_train, y_train, x_val, y_val)

# Voting

In [ ]:
def voting_models(models, x_train, y_train, x_val, y_val, cross_val):
    
    voting = VotingClassifier(estimators=models,
                                    voting='soft')

    voting.fit(x_train, y_train)

    predictions_boost = voting.predict(x_val)

    print("accuracy boost: ",  accuracy_score(y_val, predictions_boost))

    if cross_val:
        # Utiliser la validation croisée pour évaluer la performance du modèle
        scores = cross_val_score(voting, x_train, y_train, cv=5, scoring='accuracy')
        print("Cross-validation accuracy scores: ", scores)
        print("Mean cross-validation accuracy: ", scores.mean())
    
    return voting
# ex :
vot = voting_models([('xgb', xgb), ('cat', cat), ('lgb', lgb)], x_train, y_train, x_val, y_val)

# Submission

In [ ]:
def submission(model, test, Id):
    pred = model.predict(test)

    submission = pd.DataFrame({'id': Id,
                               'target': Id})

    # Save submission to a CSV file
    submission.to_csv('submission.csv', index=False)
    
submission(voting_boost, X_test, id_test)